In [1]:
import os
import sys
sys.path.append("../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
)
from utils.model_utils.save_module import save_module
from utils.model_utils.load_model import load_model
from utils.model_utils.evaluate import evaluate_model, get_sparsity, similar
from utils.dataset_utils.sampling import SamplingDataset
from utils.prune_utils.prune import (
    prune_wanda
)

In [3]:
name= "OSDG"
device = torch.device("cuda:0")
checkpoint = None
batch_size=16
num_workers=4
num_samples=4
wanda_ratio=0.3
seed=44
include_layers=["attention", "intermediate", "output"]
exclude_layers=None

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-08-20 18:02:39


In [5]:
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]
model, tokenizer, checkpoint = load_model(model_config)

Loading the model.

{'model_name': 'sadickam/sdg-classification-bert', 'task_type': 'classification', 'architectures': 'bert', 'dataset_name': 'OSDG', 'num_labels': 16, 'cache_dir': 'Models'}

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

C:\Users\Administrator\anaconda3\envs\DecomposeTransformer\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\Documents\Lab\DecomposeTransformer\Models\models--sadickam--sdg-classification-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

The model sadickam/sdg-classification-bert is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    name, batch_size=batch_size, num_workers=num_workers, do_cache=True, seed=seed
)

{'dataset_name': 'OSDG', 'path': 'albertmartinez/OSDG', 'config_name': '2024-01-01', 'text_column': 'text', 'label_column': 'labels', 'cache_dir': 'Datasets/OSDG', 'task_type': 'classification'}

Generating train split:   0%|          | 0/29844 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12791 [00:00<?, ? examples/s]

Tokenizing dataset:   0%|                                                                    | 0/29844 [00:00<?, ?it/s]

Tokenizing dataset:   1%|3                                                       | 192/29844 [00:00<00:15, 1919.21it/s]

...

Tokenizing dataset:  96%|####################################################1 | 12342/12791 [00:05<00:00, 2273.85it/s]

Tokenizing dataset:  98%|##################################################### | 12570/12791 [00:05<00:00, 2271.19it/s]

Tokenizing dataset: 100%|######################################################| 12791/12791 [00:05<00:00, 2231.51it/s]

Caching is completed.

The dataset OSDG is loaded

In [7]:
all_samples = SamplingDataset(
    train_dataloader, 200, num_samples, num_labels, False, 4, device=device, resample=False, seed=seed
)

In [8]:
# print("Evaluate the original model")
# result = evaluate_model(model, model_config, test_dataloader)

In [9]:
module = copy.deepcopy(model)
prune_wanda(module, model_config, all_samples, sparsity_ratio=wanda_ratio, include_layers=include_layers, exclude_layers=exclude_layers)
print("Evaluate the pruned model")
result = evaluate_model(module, model_config, test_dataloader)
# save_module(module, "Modules/", f"wanda_{name}_{wanda_ratio}p.pt")

C:\Users\Administrator\anaconda3\envs\DecomposeTransformer\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Evaluate the pruned model

Evaluating:   0%|                                                                              | 0/800 [00:00<?, ?it/s]

Evaluating:   0%|                                                                    | 1/800 [00:16<3:41:58, 16.67s/it]

Evaluating:   0%|▏                                                                   | 2/800 [00:16<1:32:34,  6.96s/it]

...

Evaluating: 100%|███████████████████████████████████████████████████████████████████▉| 799/800 [02:29<00:00,  5.93it/s]

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 800/800 [02:30<00:00,  5.32it/s]

Loss: 0.9430

Precision: 0.7766, Recall: 0.7823, F1-Score: 0.7753

              precision    recall  f1-score   support

           0       0.76      0.66      0.71       797
           1       0.84      0.71      0.77       775
           2       0.87      0.87      0.87       795
           3       0.87      0.83      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.68      0.77       882
           6       0.86      0.80      0.83       940
           7       0.48      0.59      0.53       473
           8       0.66      0.85      0.74       746
           9       0.59      0.73      0.66       689
          10       0.76      0.78      0.77       670
          11       0.62      0.80      0.70       312
          12       0.71      0.81      0.76       665
          13       0.84      0.85      0.85       314
          14       0.85      0.77      0.81       756
          15       0.97      0.97      0.97      1607

    accuracy                           0.80     12791
   macro avg       0.78   

In [10]:
for concern in range(num_labels):
    print(f"--{concern}--")
    positive_samples = SamplingDataset(
        train_dataloader, concern, num_samples, num_labels, True, 4, device=device, resample=False, seed=seed
    )
    negative_samples = SamplingDataset(
        train_dataloader, concern, num_samples, num_labels, False, 4, device=device, resample=False, seed=seed
    )
    similar(model, module, valid_dataloader, concern, num_samples, num_labels, device=device, seed=seed)

--0--

adding eps to diagonal and taking inverse

taking square root

dot products...

trying to take final svd

...

Linear CKA concern: 0.9854555579456422

Linear CKA non-concern: 0.987451837806061

Kernel CKA concern: 0.981791186222205

Kernel CKA non-concern: 0.9867210340439948

--13--

In [ ]:
get_sparsity(module)